In [ ]:
# pip install ddgs requests

import requests
from ddgs import DDGS

LLAMA_URL = "http://127.0.0.1:5200/v1/chat/completions"


def search_duckduckgo(query, num_results=10):
	results = []
	with DDGS() as ddgs:
		for r in ddgs.text(query, max_results=num_results):
			print(r)
			results.append({
				"title": r.get("title"),
				"snippet": r.get("body"),
				"url": r.get("href")
			})
	return results


def build_rag_context(results):
	context = "### Wyniki DuckDuckGo:\n"
	for i, r in enumerate(results, 1):
		context += (
			f"{i}. {r['title']}\n"
			f"   {r['snippet']}\n"
			f"   Źródło: {r['url']}\n\n"
		)
	return context


def send_prompt(user_query, rag=False):
	prompt = ""
	if rag:
		search_results = search_duckduckgo(user_query)

		rag_context = build_rag_context(search_results)

		prompt = (
			"Użyj poniższych informacji do odpowiedzi.\n"
			"Jeśli czegoś nie ma w wynikach, odpowiedz na podstawie własnej wiedzy.\n"
			f"{rag_context}\n"
			"### Pytanie użytkownika:\n"
			f"{user_query}\n"
		)
		print(prompt)
	else:
		prompt = user_query

	request = {
		"messages": [
			{"role": "system", "content": "Jesteś pomocnym asystentem."},
			{"role": "user", "content": prompt}
		],
		"max_tokens": -1
	}

	resp = requests.post("http://192.168.238.18:5200/v1/chat/completions", json=request)

	return resp.json()["choices"][0]["message"]["content"]

query = "What is the population of Norway?"
answer = send_prompt(query, True)
print(answer)


Użyj poniższych informacji do odpowiedzi.
Jeśli czegoś nie ma w wynikach, odpowiedz na podstawie własnej wiedzy.
### Wyniki DuckDuckGo:
1. {'title': 'Demographics of Norway - Wikipedia', 'href': 'https://en.wikipedia.org/wiki/Demographics_of_Norway', 'body': 'Demographics of Norway ... Demographic features of the population of Norway , including Jan Mayen, and Svalbard, where the hospital is not equipped for births, and no burials are allowed because of permafrost, include population density, ethnicity, education level, health of the populace, economic status, religious affiliations and other aspects.'}

2. {'title': 'Norway - Wikipedia', 'href': 'https://en.wikipedia.org/wiki/Norway', 'body': 'Norway maintains close ties with the European Union and the United States. Norway is a founding member of the United Nations, NATO, the European Free Trade Association, the Council of Europe, the Antarctic Treaty, and the Nordic Council; a member of the European Economic Area, the WTO, and the O